In [ ]:
# Fix numpy compatibility and install TensorFlow 2.20 with dependencies
print("🔧 Fixing numpy compatibility issues...")

# First, uninstall problematic packages
!pip uninstall -y numpy opencv-python tensorflow

# Install numpy first with specific version for TensorFlow 2.20
!pip install numpy==1.26.4

# Install TensorFlow 2.20 (it will use compatible numpy)
!pip install tensorflow==2.20.0

# Install other dependencies with compatible versions
!pip install opencv-python==4.10.0.84
!pip install pillow==10.4.0
!pip install matplotlib==3.8.4

print("🔄 Restarting Python kernel is recommended after installation...")
print("📋 Please restart the kernel and then run the verification cell below.")


In [ ]:
# Verification Cell - Run this AFTER restarting the kernel
print("🔍 Verifying installations...")

try:
    import numpy as np
    print(f"✅ NumPy {np.__version__} - OK")
except Exception as e:
    print(f"❌ NumPy error: {e}")

try:
    import tensorflow as tf
    print(f"✅ TensorFlow {tf.__version__} - OK")
except Exception as e:
    print(f"❌ TensorFlow error: {e}")

try:
    import cv2
    print(f"✅ OpenCV {cv2.__version__} - OK")
except Exception as e:
    print(f"❌ OpenCV error: {e}")

try:
    import PIL
    print(f"✅ PIL {PIL.__version__} - OK")
except Exception as e:
    print(f"❌ PIL error: {e}")

try:
    import matplotlib
    print(f"✅ Matplotlib {matplotlib.__version__} - OK")
except Exception as e:
    print(f"❌ Matplotlib error: {e}")

print("\n🎯 If all packages show ✅, you're ready to proceed!")
print("📝 If any show ❌, please run the installation cell again.")

In [ ]:
# Import with error handling and modern TensorFlow APIs
try:
    import tensorflow as tf
    from tensorflow.keras.applications.resnet50 import preprocess_input
    # Use tf.keras.utils.load_img instead of deprecated preprocessing.image
    from tensorflow.keras.utils import load_img, img_to_array
    import numpy as np
    print(f"✅ TensorFlow {tf.__version__} loaded successfully")
    print("✅ Using modern tf.keras APIs")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please run the compatibility fix cell above and restart the kernel")
    raise

# Load model with error handling
try:
    model = tf.keras.models.load_model('saved_models/waste_classifier_balanced.keras')
    print("✅ Model loaded successfully")
    print(f"Model input shape: {model.input_shape}")
    print(f"Model output shape: {model.output_shape}")
except Exception as e:
    print(f"❌ Model loading error: {e}")
    print("Please ensure the model file exists and TensorFlow is properly installed")
    raise

def predict_waste(img_path):
    """Predict waste type from image path using modern tf.keras APIs"""
    try:
        # Use modern tf.keras.utils functions
        img = load_img(img_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        
        prediction = model.predict(img_array, verbose=0)[0][0]
        
        if prediction > 0.5:
            return f"Non-biodegradable (confidence: {prediction:.2%})"
        else:
            return f"Biodegradable (confidence: {1-prediction:.2%})"
    except Exception as e:
        return f"Error predicting: {e}"

print("✅ Prediction function ready!")
print("Example usage:")
print("result = predict_waste('dataset/mt/t1.webp')")
print("print(result)")

# Test if we can make a sample prediction (uncomment to test)
# try:
#     result = predict_waste('dataset/mt/t1.webp')
#     print(f"Test result: {result}")
# except Exception as e:
#     print(f"Test failed: {e}")

In [ ]:
# Real-time Waste Classification using Laptop Camera (Updated with modern APIs)
import cv2
import numpy as np
from PIL import Image
import time

class RealTimeWasteClassifier:
    def __init__(self, model_path):
        try:
            self.model = tf.keras.models.load_model(model_path)
            print(f"✅ Real-time classifier model loaded: {model_path}")
        except Exception as e:
            print(f"❌ Failed to load real-time model: {e}")
            raise
        
        self.cap = None
        self.classification_interval = 1.0  # Classify every 1 second
        self.last_classification_time = 0
        self.current_prediction = "Ready to classify..."
        self.confidence = 0.0
        
    def preprocess_frame(self, frame):
        """Preprocess camera frame for model prediction using modern APIs"""
        try:
            # Convert BGR to RGB
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Convert to PIL Image and resize
            pil_image = Image.fromarray(rgb_frame)
            pil_image = pil_image.resize((224, 224))
            
            # Convert to array and preprocess
            img_array = np.array(pil_image, dtype=np.float32)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            
            return img_array
        except Exception as e:
            print(f"Frame preprocessing error: {e}")
            return None
    
    def predict_frame(self, frame):
        """Predict waste type from camera frame"""
        try:
            processed_frame = self.preprocess_frame(frame)
            if processed_frame is None:
                return "Error", 0.0, (255, 255, 255)
                
            prediction = self.model.predict(processed_frame, verbose=0)[0][0]
            
            if prediction > 0.5:
                waste_type = "Non-biodegradable"
                confidence = prediction
                color = (0, 0, 255)  # Red for non-biodegradable
            else:
                waste_type = "Biodegradable"
                confidence = 1 - prediction
                color = (0, 255, 0)  # Green for biodegradable
            
            return waste_type, confidence, color
        except Exception as e:
            print(f"Prediction error: {e}")
            return "Error", 0.0, (255, 255, 255)
    
    def add_text_overlay(self, frame, waste_type, confidence, color):
        """Add prediction overlay to frame"""
        overlay = frame.copy()
        cv2.rectangle(overlay, (10, 10), (500, 100), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
        
        # Add main prediction text
        cv2.putText(frame, f"Prediction: {waste_type}", 
                   (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        
        # Add confidence text
        cv2.putText(frame, f"Confidence: {confidence:.1%}", 
                   (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # Add instructions
        cv2.putText(frame, "Press 'q' to quit, 's' to save frame", 
                   (10, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return frame
    
    def start_camera(self):
        """Start real-time camera classification"""
        print("Starting camera... Press 'q' to quit, 's' to save current frame")
        
        # Initialize camera
        self.cap = cv2.VideoCapture(0)
        
        if not self.cap.isOpened():
            print("Error: Could not open camera")
            return
        
        # Set camera properties
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        
        frame_count = 0
        
        try:
            while True:
                ret, frame = self.cap.read()
                if not ret:
                    print("Error: Could not read frame")
                    break
                
                # Flip frame horizontally for mirror effect
                frame = cv2.flip(frame, 1)
                
                # Classify frame periodically
                current_time = time.time()
                if current_time - self.last_classification_time >= self.classification_interval:
                    waste_type, confidence, color = self.predict_frame(frame)
                    self.current_prediction = waste_type
                    self.confidence = confidence
                    self.color = color
                    self.last_classification_time = current_time
                
                # Add overlay with current prediction
                if hasattr(self, 'color'):
                    frame = self.add_text_overlay(frame, self.current_prediction, 
                                                self.confidence, self.color)
                
                # Display frame
                cv2.imshow('Real-time Waste Classification', frame)
                
                # Handle key presses
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('s'):
                    # Save current frame
                    filename = f"classified_frame_{int(time.time())}.jpg"
                    cv2.imwrite(filename, frame)
                    print(f"Frame saved as {filename}")
                
                frame_count += 1
                
        except KeyboardInterrupt:
            print("\nStopping camera...")
        
        finally:
            self.cap.release()
            cv2.destroyAllWindows()
            print("Camera stopped.")

# Initialize the real-time classifier
classifier = RealTimeWasteClassifier('saved_models/waste_classifier_balanced.keras')

print("Real-time Waste Classifier Ready!")
print("Instructions:")
print("- Run classifier.start_camera() to begin")
print("- Point camera at waste items")
print("- Press 'q' to quit")
print("- Press 's' to save current frame")

In [ ]:
# Start Real-time Classification
# Run this cell to start the camera classification

# Start the camera (uncomment the line below to run)
# classifier.start_camera()

# Alternative: Test with a specific region of interest
def start_camera_with_roi():
    """Start camera with a region of interest box for better classification"""
    print("Starting camera with ROI... Press 'q' to quit")
    
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open camera")
        return
    
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    # ROI coordinates (center square)
    roi_size = 200
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame = cv2.flip(frame, 1)
            h, w = frame.shape[:2]
            
            # Calculate ROI coordinates
            roi_x1 = w//2 - roi_size//2
            roi_y1 = h//2 - roi_size//2
            roi_x2 = roi_x1 + roi_size
            roi_y2 = roi_y1 + roi_size
            
            # Extract ROI
            roi = frame[roi_y1:roi_y2, roi_x1:roi_x2]
            
            # Classify ROI
            if roi.size > 0:
                try:
                    # Convert ROI to RGB
                    rgb_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
                    pil_image = Image.fromarray(rgb_roi)
                    pil_image = pil_image.resize((224, 224))
                    
                    img_array = np.array(pil_image)
                    img_array = np.expand_dims(img_array, axis=0)
                    img_array = preprocess_input(img_array)
                    
                    prediction = classifier.model.predict(img_array, verbose=0)[0][0]
                    
                    if prediction > 0.5:
                        waste_type = "Non-biodegradable"
                        confidence = prediction
                        color = (0, 0, 255)  # Red
                    else:
                        waste_type = "Biodegradable"
                        confidence = 1 - prediction
                        color = (0, 255, 0)  # Green
                    
                    # Draw ROI box
                    cv2.rectangle(frame, (roi_x1, roi_y1), (roi_x2, roi_y2), color, 3)
                    
                    # Add text
                    cv2.putText(frame, f"{waste_type} ({confidence:.1%})", 
                               (roi_x1, roi_y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                    
                except Exception as e:
                    # Draw white ROI box on error
                    cv2.rectangle(frame, (roi_x1, roi_y1), (roi_x2, roi_y2), (255, 255, 255), 2)
                    cv2.putText(frame, "Processing...", 
                               (roi_x1, roi_y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # Instructions
            cv2.putText(frame, "Place waste item in the box", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(frame, "Press 'q' to quit", 
                       (10, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            cv2.imshow('Waste Classification - ROI Mode', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    except KeyboardInterrupt:
        print("\nStopping camera...")
    finally:
        cap.release()
        cv2.destroyAllWindows()
        print("Camera stopped.")

print("Camera functions ready!")
print("\nOptions to start:")
print("1. classifier.start_camera()          # Full screen classification")
print("2. start_camera_with_roi()            # Region of interest mode")
print("\nRecommended: Use option 2 for better accuracy!")

In [ ]:
# start_camera_with_roi()
classifier.start_camera()

In [ ]:
import tensorflow as tf
print(tf.__version__)